In [1]:
'''
Aim: To visualize SWMF 3d data with pyvista volume rendering.
Modified for OH Data
Revised 23/03/10
'''

import numpy as np
import matplotlib.pyplot as plt
import imageio
import pyvista
from spacepy.pybats import IdlFile

# from plot_SWMF_Bxyz import Trace_in_box
k_b = 1.38e-23



def Trace_in_box(data_box, src_radius=100, n_src_points=100,max_time=1000.):
    # r,lon,lat = np.array(data_shl['r']),np.deg2rad(np.array(data_shl['lon'])),np.deg2rad(np.array(data_shl['lat']))
    x, y, z = np.array(data_box['x']), np.array(data_box['y']), np.array(data_box['z'])
    Bx, By, Bz = np.array(data_box['Bx']), np.array(data_box['By']), np.array(data_box['Bz'])

    xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')

    mesh = pyvista.StructuredGrid(xv, yv, zv)

    vectors = np.empty((mesh.n_points, 3))
    vectors[:, 0] = Bx.ravel(order='F')
    vectors[:, 1] = By.ravel(order='F')
    vectors[:, 2] = Bz.ravel(order='F')
    mesh['vectors'] = vectors
    streams = []
    stream, src = mesh.streamlines('vectors', return_source=True, source_radius=src_radius, n_points=n_src_points,
                                   progress_bar=True,
                                   integration_direction='both', max_time=max_time)
    streams.append(stream)

    return stream

def Trace_in_box_from_targets(data_box, pos_target,max_time=1000.,direction='both'):

    x, y, z = np.array(data_box['x']), np.array(data_box['y']), np.array(data_box['z'])
    Bx, By, Bz = np.array(data_box['Bx']), np.array(data_box['By']), np.array(data_box['Bz'])

    xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')

    mesh = pyvista.StructuredGrid(xv, yv, zv)

    vectors = np.empty((mesh.n_points, 3))
    vectors[:, 0] = Bx.ravel(order='F')
    vectors[:, 1] = By.ravel(order='F')
    vectors[:, 2] = Bz.ravel(order='F')
    mesh['vectors'] = vectors
    # streams = []
    # stream, src = mesh.streamlines('vectors', return_source=True, source_radius=src_radius, n_points=n_src_points,
    #                                progress_bar=True,
    #                                integration_direction='both', max_time=max_time)
    # streams.append(stream)
    streams = []

    for i in range(len(pos_target)):
        start_point = pos_target[i]
        stream = mesh.streamlines('vectors', progress_bar=True, start_position=start_point, return_source=False,
                              integration_direction=direction,
                              max_time=max_time)
        streams.append(stream)

    return streams

if __name__ == '__main__':
    # %%
    data_path = '/Users/ephe/THL8/OH_THC/'
    file_type = 'box_mhd_5_'
    n_iter = 20000
    n_time = None

    filename = file_type + 'n' + str(int(n_iter)).zfill(8)

    print('Reading file: '+data_path + filename + '.out')
    data_box = IdlFile(data_path + filename + '.out')
    var_list = list(data_box.keys())
    unit_list = data_box.meta['header'].split()[1:]
    print('Variables: ', var_list)
    print('Units: ', unit_list)
    x = data_box['x']
    y = data_box['y']
    z = data_box['z']
    dimensions = data_box['grid']
    spacing = (abs(x[1] - x[0]), abs(y[1] - y[0]), abs(z[1] - z[0]))
    origin = (x[0], y[0], z[0])
    print(origin)
    print(spacing)

#     data_rho = data_box['Rho']
#     data_p = data_box['P']
#     data_T = (data_p / 10) / k_b / (2 * data_rho * 1e6)
#     var_str = 'T [K]'
#     plot_data = data_T

    var_str = 'Rho [amu/cc]'
    plot_data = np.array(data_box['Rho'])
    
    cmin = plot_data.min()
    cmax = plot_data.max()
    print(cmin,cmax)

    box_grid = pyvista.UniformGrid(dimensions=(dimensions[0], dimensions[1], dimensions[2]), spacing=spacing,
                                   origin=origin)
    box_grid.point_data[var_str] = plot_data.ravel('F')


Reading file: /Users/ephe/THL8/OH_THC/box_mhd_5_n00020000.out
Variables:  ['grid', 'x', 'y', 'z', 'Rho', 'Ux', 'Uy', 'Uz', 'Bx', 'By', 'Bz', 'P', 'NeuRho', 'NeuUx', 'NeuUy', 'NeuUz', 'NeuP', 'Ne2Rho', 'Ne2Ux', 'Ne2Uy', 'Ne2Uz', 'Ne2P', 'Ne3Rho', 'Ne3Ux', 'Ne3Uy', 'Ne3Uz', 'Ne3P', 'Ne4Rho', 'Ne4Ux', 'Ne4Uy', 'Ne4Uz', 'Ne4P', 'jx', 'jy', 'jz']
Units:  ['AU', 'AU', 'amu/cc', 'km/s', 'km/s', 'km/s', 'nT', 'nT', 'nT', 'dyne/cm^2', 'amu/cc', 'km/s', 'km/s', 'km/s', 'dyne/cm^2', 'amu/cc', 'km/s', 'km/s', 'km/s', 'dyne/cm^2', 'amu/cc', 'km/s', 'km/s', 'km/s', 'dyne/cm^2', 'amu/cc', 'km/s', 'km/s', 'km/s', 'dyne/cm^2', 'uA/m^2', 'uA/m^2', 'uA/m^2']
(-300.0, -600.0, -600.0)
(5.0, 12.0, 12.0)
0.0 0.09194774


In [8]:
    # stream_outer = Trace_in_box(data_box,src_radius=300.,n_src_points=100,max_time=1000.)
    # stream_inner = Trace_in_box(data_box,src_radius=50.,n_src_points=100,max_time=1000.)

Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]


In [81]:
    n_point = 80
    latitudes = np.deg2rad(np.random.uniform(low=-90., high=90., size=n_point))
    longitudes = np.deg2rad(np.random.uniform(low=0., high=360, size=n_point))
    radius = 300.
    start_points = [[radius*np.cos(latitudes[i])*np.cos(longitudes[i]), radius*np.cos(latitudes[i])*np.sin(longitudes[i]), radius*np.sin(latitudes[i])] for i in range(n_point)]

    pos_target = start_points
    stream_outer = Trace_in_box_from_targets(data_box,pos_target,max_time=5000.,direction='both')



Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<

In [103]:
    n_point_inner = 11
    latitudes = np.deg2rad(np.random.uniform(low=-75., high=75., size=n_point_inner))
    latitudes = np.deg2rad([-80,-60,-45,-30,-15,0,15,30,45,60,80,])
    longitudes = np.deg2rad(np.random.uniform(low=0., high=360, size=n_point_inner))
    radius = 80.
    start_points = [[radius*np.cos(latitudes[i])*np.cos(longitudes[i]), radius*np.cos(latitudes[i])*np.sin(longitudes[i]), radius*np.sin(latitudes[i])] for i in range(n_point_inner)]

    pos_target = [[50,0,0],[50,0,50]]
    pos_target = start_points
    stream_inner = Trace_in_box_from_targets(data_box,pos_target,max_time=10000.,direction='backward')

Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]
Generating Streamlines: 100%|██████████[00:00<00:00]


In [1]:
    pyvista.jupyter_backend='panel'
    p = pyvista.Plotter(notebook=True,window_size=(1024,768))

    vol = p.add_volume(box_grid, cmap='magma',
                       clim=[1e-4, 1e-2],  # Change Colorbar
                       opacity=(0., .5, 0.),  # Change Opacity Mask
                       opacity_unit_distance=150)  # 调整透明的程度

    for i in range(n_point):
        p.add_mesh(stream_outer[i].tube(radius=1.), color='green')
    for i in range(n_point_inner):
        p.add_mesh(stream_inner[i].tube(radius=1.), color='silver')

    p.set_background("white")

    # p.show_grid(xlabel='X [AU]', ylabel='Y [AU]', zlabel='Z [AU]',font_size=10) #!!!!!这个命令不起作用
    p.add_axes(xlabel='X',ylabel='Y',zlabel='Z')
    p.export_html(data_path + 'sample.html',backend='panel')
    p.show(jupyter_backend='panel')



NameError: name 'pyvista' is not defined